# Funciones para encontrar usuarios a testear

In [1]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from tqdm import tqdm
import pickle

In [4]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

In [5]:
# targetUsers = pd.DataFrame(columns=['username'])

targetUsers = pd.read_pickle('../datasets/targetUsersDataset.pkl')
print(len(targetUsers))
targetUsers.head()

15805


,username
235491971,gsrcaballero
497428913,juannfalopa
1536923558,FabianPereyra87
2300560809,NMoraniB
179471208,Sergioogz


In [17]:
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getNewPage(term, searchType, maxPosition):
    if(searchType != 'q'):
        term = '#'+term
    pageResponse = requests.get(
        'https://twitter.com/i/search/timeline',
        params={
            'vertical': 'news',
            'q': term,
            'max_position': maxPosition,
            'include_entities': 1,
            'include_available_features': 1,
            'reset_error_state': False
        },
        proxies=proxies,
        headers=headersMore
    )
    return(pageResponse.json())
#     print(pageResponse.json().keys())
#     print(pageResponse.json()['min_position'])
#     display(BeautifulSoup(pageResponse.json()['items_html'], 'html.parser'))
    return pageResponse.json()['min_position'], BeautifulSoup(pageResponse.json()['items_html'], 'html.parser')

def getUserTweetData(tweet):
    try:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userId = tweet.findAll('div', attrs={'class':'js-stream-tweet'})[0].attrs['data-user-id']
        targetUsers.loc[userId] = [username]
    except:
        print('ERROR---------------------')
        print(tweet)
    return [username, userId]

def searchTerm(term, searchType='q', maxPages=15):
    pageResponse = False
    if(searchType == 'q'):
        pageResponse = requests.get('https://twitter.com/search?q='+term, headers=headersChrome, proxies=proxies)
    else:
        pageResponse = requests.get('https://twitter.com/hashtag/'+term, headers=headersChrome, proxies=proxies)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)
    for tweet in tweetList:
        getUserTweetData(tweet)
    nextKey = dataMax
    for pageNum in tqdm(range(maxPages)):
        response = getNewPage(term, searchType, nextKey)
        nextKey = response['min_position']
        htmlResponse = BeautifulSoup(response['items_html'], 'html.parser')
        if(len(htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)) == 0):
            print(response)
            print('NO MORE',htmlResponse)
            break
        for tweet in htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False):
            getUserTweetData(tweet)

# searchTerm('cfk', '#', 15)
# searchTerm('cambiemos', '#', 5)
# searchTerm('MacriVendepatria', '#', 5)
# searchTerm('JuntosPorElCambio', '#', 5)
# searchTerm('carrio', 'q', 100)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.35s/it]


In [19]:
def process_id(id):
    """process a single ID"""
    # fetch the data
    r = requests.get(url_t % id)
    # parse the JSON reply
    data = r.json()
    # and update some data with PUT
    requests.put(url_t % id, data=data)
    return data

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = searchTerm(id, 'q', 600)
    return store

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    print('Done')
    print(len(targetUsers))
    display(targetUsers.head())
    return 'Done'

termsToSearch = [
    'cfk',
    'MacriVendepatria',
    'yovotoalfrentedetodos',
    'ElijoALosFernandez',
    'FernandezFernandez2019',
    'macriespobreza',
    'macrigato',
    'FueraMacri',
    
    'cambiemos',
    'JuntosPorElCambio',
    'DefensoresDelCambio',
    'SigamosCambiandoJuntos',
    'ListaCompletaDelCambio',
    'MacriPichetto2019',
    'NoVuelvenMas',
    'EllaMiente'
]
termsToSearch = [
    'macri',
    'vidal',
    'macri gato',
    'macri pobreza',
    'macri voto',
    'vidal voto',
    'cistina voto',
    'fernandez voto',
    'macri inflacion',
    'vidal inflacion',
    'cistina inflacion',
    'fernandez inflacion',
    'elecciones 2019',
]
threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)
    
# termsToSearch = [
#     'macri FMI',
#     'sinceramente falso',
#     'ella miente',
#     'massa',
#     'cristina pobreza'
# ]
# threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)

  0%|                                                                                          | 0/600 [00:00<?, ?it/s]



ERROR---------------------
ERROR---------------------ERROR---------------------

  0%|                                                                                          | 0/600 [00:00<?, ?it/s]



<li class="js-stream-item stream-item stream-item " data-item-id="1146106408691089408" data-item-type="tweet" id="stream-item-tweet-1146106408691089408">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content " data-component-context="tweet" data-conversation-id="1146106408691089408" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1146106408691089408" data-name="Popular" data-permalink-path="/populardiario/status/1146106408691089408" data-reply-to-users-json='[{"id_str":"227476236","screen_name":"populardiario","name":"Popular","emojified_name":{"text":"Popular","emojified_text_as_html":"Popular"}}]' data-screen-name="populardiario" data-tweet-id="1146106408691089408" data-tweet-nonce="1146106408691089408-80183cbc-9bfb-4425-890b-6a08cea07551" data-tweet-stat-initialized="true" data-user-id="227476236" data-you-block="false" data-you-follow="false"

</li><li class="js-stream-item stream-item stream-item " data-item-id="1146078562103025664" data-item-type="tweet" id="stream-item-tweet-1146078562103025664">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content " data-component-context="tweet" data-conversation-id="1146078562103025664" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1146078562103025664" data-name="Flamengo" data-permalink-path="/Flamengo/status/1146078562103025664" data-reply-to-users-json='[{"id_str":"59591856","screen_name":"Flamengo","name":"Flamengo","emojified_name":{"text":"Flamengo","emojified_text_as_html":"Flamengo"}}]' data-screen-name="Flamengo" data-tweet-id="1146078562103025664" data-tweet-nonce="1146078562103025664-1dd90471-1e3b-4df5-b746-5d165b18e626" data-user-id="59591856" data-you-block="false" data-you-follow="false">
<div class="context">
</div>
<div class="co

...

{'min_position': 'thGAVUV0VFVBbK9t2BRRaMwLWpodWC6B8SGMgFBD1UO2FFu9QAAAAAAAAVfAAAAAcAAABWAaRgkCIgKNCAJZJASiEAEI2BLQcCCBGoAQIgPCUIRYEhDAAkLqfgAAWo5ECjKVIDABBQAABVIeQCJIQAQIACTAQIJQVAwRByAAUCyEgAAALECEpiIgyBABBURAIgAGDFFUfgCEAAAKAQGBKcxXBABAKP6IsoBEyIABZnAwMBB7gwKCM0AggDQwAoIaBWgkBCAAPBVASMQPsABDMgIJgIgADAhgIgggJAHhhloFoABDYAEIEsUrBgCuybAYICDECFQRBAKQoEBQghEAiCeWQgFgGIABbgYNkiDIICSAAMAQUgVEYpJChJAC4giDgAgBWRyEAxAAgIWICEDYQIAgCHRAM7EFzSBIoEeSAkAGEYSkmrQuoJAIVgASBECgIAwcgQCtIAABQIQDIhp8RkKEvTUjIS6GBUEIJgAJAJpIagEwEaVCEEQAAIAQBgDADBBAQBKEhJEnbhgACBFAF4JoEENDA72AAk8knQKKKDECBBAEaWMQAcBAAJUgAMYAEoQQSRdRgEDIERhmIKC0BAJAIQAcIQsCBEQAIiAGVAEiBpBXMoAs5iIMokCyYReqAEEEAikDBoAAUEgAAQ2MMCAQCiuofAgAgigEBiGKAGABEgNCYTSKQgwnSNXQFQJcRJIkAkAKwACGmoMIsYCAABAQGGpyEhZHAtowgkARICwCCNALREEAwIkECABigYAAAJGAAKQwJEBDRAjCAD0EQgIITAiAgEBiAUIhwlDA4gCVAVCAQRAcCIJhAkUIDDAAIMAlJQoA4AgNIASaRECBAEMCUCZBInmqCEBDAKQiIShIgpowBSFBCZDBsCKII2WUBHCiRkFcEWUCGSQxgAQ0EQAEpMC5MIkhwkAQICABYMCHA4QSAMRCCDBAAABTKSGgojDMRAMCgRDIEgwa4GRQjB












  0%|                                                                                          | 0/600 [00:00<?, ?it/s]

{'min_position': 'thGAVUV0VFVBYBFoSAsLWlkK6HHxIYTBJjwusAAAH0P4BiTdLxqfwAAAAHBENYMf7CQAAPh1xBK1YAAg7pVaZ41tABDlU8f94X4AAOyvmtOFawAAYGTyaQwmAAA9t5Q7fCcAAVAhUAFQAlAFUAFQAA', 'has_more_items': False, 'items_html': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n', 'new_latent_count': 0, 'focused_refresh_interval': 30000}
NO MORE 

Done
17205


,username
235491971,gsrcaballero
497428913,juannfalopa
1536923558,FabianPereyra87
2300560809,NMoraniB
179471208,Sergioogz


'Done'

In [20]:
len(targetUsers)

17205

In [23]:
#guardar resultados
import pickle
filename = '../datasets/targetUsersDataset.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)